# 🌍 Kit de Herramientas de Geocodificación - Interfaz Amigable

**¡Bienvenido!** Este notebook te permitirá procesar archivos de coordenadas y direcciones de manera sencilla **desde cualquier ubicación** en tu computadora.

## ¿Qué puedes hacer aquí?
- 📍 **Extraer coordenadas** de archivos KML (Google Earth, BatchGeo)
- 🔄 **Geocodificación inversa** (coordenadas → direcciones)
- ✨ **Limpiar y enriquecer** datos de direcciones
- 📊 **Exportar resultados** en múltiples formatos (CSV, Excel, JSON)

## 💻 Compatible con cualquier ubicación:
- 📁 **Archivos locales**: `C:\MisCarpetas\datos.csv`
- ☁️ **OneDrive**: `C:\Users\Usuario\OneDrive\Documentos\archivo.csv`
- 🌐 **OneDrive Empresarial**: `C:\Users\Usuario\OneDrive - Empresa\datos.kml`
- 💾 **Unidades externas**: `D:\MisArchivos\coordenadas.txt`
- 🗂️ **Carpetas compartidas**: `\\Servidor\Compartida\datos.csv`

## Instrucciones para principiantes:
1. Ejecuta cada celda en orden (Shift + Enter)
2. Usa el explorador de archivos integrado o pega rutas completas
3. ¡No te preocupes si eres nuevo en esto!

### 🎯 Formatos de archivo soportados:
- **CSV**: Archivos con coordenadas en columnas (Excel compatible)
- **TXT**: Coordenadas separadas por comas o espacios
- **KML**: Archivos de Google Earth, Google Maps, BatchGeo
- **JSON**: Datos geográficos en formato JSON

---

## 📦 Paso 1: Instalación de Dependencias

Primero instalaremos automáticamente todas las librerías necesarias.

## 💡 Guía Rápida: Encontrar Rutas de Archivos en Windows

### 🔍 ¿Cómo encontrar la ruta completa de tu archivo?

**Método 1: Desde el Explorador de Windows**
1. Abre el Explorador de Windows
2. Navega hasta tu archivo
3. Haz clic derecho → "Copiar como ruta"
4. Pega la ruta en el notebook

**Método 2: Desde la barra de direcciones**
1. Haz clic en la barra de direcciones del Explorer
2. Copia la ruta que aparece
3. Añade `\NombreDelArchivo.extensión` al final

**Método 3: Usando el explorador integrado**
- Usa el botón "📁 Explorar..." en este notebook
- ¡Se abrirá un diálogo para seleccionar archivos!

### 📂 Ubicaciones típicas de OneDrive:
```
OneDrive Personal:
C:\Users\TuUsuario\OneDrive\Documentos\

OneDrive Empresarial (ejemplo Rainmaker):
C:\Users\TuUsuario\OneDrive - Rainmaker Group\Documentos\

OneDrive Compartido:
C:\Users\TuUsuario\OneDrive - NombreEmpresa\Carpeta\
```

### ✅ Ejemplos de rutas válidas:
- `C:\Users\Juan\OneDrive\Documentos\coordenadas.csv`
- `D:\MisProyectos\datos_gps.txt`
- `C:\Users\Maria\Desktop\ubicaciones.kml`
- `\\SERVIDOR\Compartida\datos\puntos.csv`

---

In [ ]:
# Ubicaciones comunes sugeridas (sin escaneo automatico)
import os
from pathlib import Path

print("Ubicaciones Comunes Sugeridas")
print("=" * 50)

user_home = Path.home()
current_user = os.getenv('USERNAME', 'Usuario')

# Solo mostrar ubicaciones (sin verificar contenido para mayor velocidad)
common_locations = [
    ("Escritorio", user_home / "Desktop"),
    ("Documentos", user_home / "Documents"), 
    ("Descargas", user_home / "Downloads"),
    ("OneDrive Personal", user_home / "OneDrive"),
    ("OneDrive Empresarial", user_home / f"OneDrive - {os.getenv('USERDOMAIN', 'Empresa')}"),
    ("Directorio actual", Path.cwd())
]

print("Puedes usar cualquiera de estas ubicaciones:")
for name, path in common_locations:
    if path.exists():
        print(f"  {name}: {path}")

print(f"\nPara tu usuario ({current_user}):")
print("  • Copia cualquier ruta de arriba")
print("  • O usa el explorador grafico del siguiente paso")
print("  • O pega directamente la ruta completa de tu archivo")

print("\n" + "=" * 50)

In [ ]:
# Instalar dependencias automáticamente
import subprocess
import sys

def install_package(package):
    """Instala un paquete usando pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} instalado exitosamente")
        return True
    except subprocess.CalledProcessError:
        print(f"✗ Error instalando {package}")
        return False

print("🔧 Instalando dependencias necesarias...")
print("(Esto puede tomar unos minutos la primera vez)\n")

packages = [
    "python-dotenv",
    "opencage", 
    "pandas",
    "requests",
    "ipywidgets",
    "plotly",
    "folium"
]

all_success = True
for package in packages:
    if not install_package(package):
        all_success = False

print("\n" + "="*50)
if all_success:
    print("✅ ¡Todas las dependencias instaladas correctamente!")
else:
    print("⚠️  Algunas dependencias no se pudieron instalar.")
    print("Por favor, instala manualmente los paquetes faltantes.")
print("="*50)

## 📚 Paso 2: Importar Librerías y Herramientas

Ahora cargaremos todas las herramientas que vamos a usar.

In [ ]:
# Importar librerías estándar
import os
import sys
import pandas as pd
import json
import time
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Importar librerías para interfaz
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual

# Importar librerías para visualización
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Importar nuestras herramientas personalizadas
# Añadir el directorio de herramientas al path
tool_paths = [
    r'reverse_geocoding_batch_processor',
    r'.'
]

for path in tool_paths:
    if os.path.exists(path) and path not in sys.path:
        sys.path.insert(0, path)

try:
    # Importar nuestras herramientas
    from reverse_geocoding_batch import *
    from core.kml_extractor import KMLExtractor
    from address_enhancer import AddressEnhancer
    
    print("✅ Todas las herramientas cargadas correctamente")
except ImportError as e:
    print(f"⚠️  Error importando herramientas: {e}")
    print("Asegúrate de que los archivos estén en el directorio correcto")

print("\n🎯 ¡Listo para comenzar!")

## 🔑 Paso 3: Configuración de API de OpenCage

Para usar las funciones de geocodificación, necesitas una clave API de OpenCage.

### ¿Cómo obtener tu API key?
1. Ve a [OpenCage Geocoding](https://opencagedata.com/)
2. Regístrate gratis (2,500 consultas/día gratuitas)
3. Copia tu API key
4. Pégala en el campo de abajo

In [ ]:
# Widget para configurar API key
api_key_widget = widgets.Password(
    value='',
    placeholder='Pega aquí tu API key de OpenCage',
    description='API Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

save_button = widgets.Button(
    description='💾 Guardar API Key',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

status_output = widgets.Output()

def save_api_key(b):
    with status_output:
        clear_output()
        api_key = api_key_widget.value.strip()
        
        if not api_key:
            print("❌ Por favor, ingresa tu API key")
            return
            
        # Guardar en archivo .env
        env_content = f"OPENCAGE_API_KEY={api_key}\n"
        
        try:
            with open('.env', 'w') as f:
                f.write(env_content)
            
            # También establecer como variable de entorno para esta sesión
            os.environ['OPENCAGE_API_KEY'] = api_key
            
            print("✅ API key guardada correctamente")
            print("🔐 Tu clave está segura en el archivo .env")
            
            # Probar la conexión
            try:
                from opencage.geocoder import OpenCageGeocode
                geocoder = OpenCageGeocode(api_key)
                result = geocoder.geocode("Madrid, España", limit=1)
                if result:
                    print("🌍 Conexión con OpenCage exitosa")
                else:
                    print("⚠️  La API key parece válida pero la respuesta está vacía")
            except Exception as e:
                print(f"❌ Error probando la API: {e}")
                
        except Exception as e:
            print(f"❌ Error guardando API key: {e}")

save_button.on_click(save_api_key)

print("🔑 Configuración de API Key")
print("Necesitas una clave API gratuita de OpenCage para continuar.\n")

display(widgets.VBox([
    api_key_widget,
    save_button,
    status_output
]))

## 📂 Paso 4: Cargar Archivos desde Cualquier Ubicación

Aquí puedes cargar archivos desde cualquier ubicación: tu disco local, OneDrive, carpetas compartidas, etc.

### 💡 Opciones para cargar archivos:
1. **Ruta completa**: Escribe o pega la ruta completa del archivo
2. **Explorar directorio**: Ve archivos en una carpeta específica
3. **Arrastrar y soltar**: (próximamente) Arrastra archivos al notebook

In [ ]:
# Cargador de archivos simple y directo (sin escaneos lentos)
import tkinter as tk
from tkinter import filedialog
import threading

print("Cargador de Archivos Simple")
print("Dos opciones rapidas para cargar tu archivo:")
print("1. Explorar: Usa el boton para navegar")
print("2. Pegar Ruta: Copia y pega la ruta completa")
print()

# Widget para ruta del archivo
file_path_widget = widgets.Text(
    value='',
    placeholder='C:\\Users\\Usuario\\OneDrive\\Documentos\\archivo.csv',
    description='Ruta del archivo:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='700px')
)

# Boton para explorar archivos
browse_button = widgets.Button(
    description='Explorar Archivos',
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

# Boton para validar archivo
validate_button = widgets.Button(
    description='Validar Archivo',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

# Output para mostrar informacion
file_validation_output = widgets.Output()

def browse_file():
    """Abrir dialogo para seleccionar archivo"""
    def file_dialog():
        try:
            root = tk.Tk()
            root.withdraw()
            root.lift()
            root.attributes('-topmost', True)
            
            # Tipos de archivo soportados
            filetypes = [
                ('Archivos de datos', '*.csv;*.txt;*.kml;*.json'),
                ('Archivos CSV', '*.csv'),
                ('Archivos de texto', '*.txt'),
                ('Archivos KML', '*.kml'),
                ('Archivos JSON', '*.json'),
                ('Todos los archivos', '*.*')
            ]
            
            filename = filedialog.askopenfilename(
                title="Seleccionar archivo de coordenadas",
                filetypes=filetypes
            )
            
            if filename:
                file_path_widget.value = filename
                validate_file()
            
            root.destroy()
            
        except Exception as e:
            with file_validation_output:
                clear_output()
                print(f"Error abriendo explorador: {e}")
                print("Puedes escribir la ruta manualmente")
    
    # Ejecutar en thread separado
    thread = threading.Thread(target=file_dialog)
    thread.daemon = True
    thread.start()

def validate_file():
    """Validar archivo seleccionado"""
    with file_validation_output:
        clear_output()
        
        file_path = file_path_widget.value.strip()
        if not file_path:
            print("Especifica la ruta de un archivo o usa 'Explorar Archivos'")
            return
            
        try:
            file = Path(file_path)
            
            if not file.exists():
                print(f"Archivo no encontrado: {file_path}")
                print("Verifica que la ruta sea correcta")
                return
                
            if not file.is_file():
                print(f"La ruta no es un archivo: {file_path}")
                return
            
            # Informacion basica del archivo
            size = file.stat().st_size
            if size < 1024:
                size_str = f"{size} bytes"
            elif size < 1024 * 1024:
                size_str = f"{size/1024:.1f} KB"
            else:
                size_str = f"{size/(1024*1024):.1f} MB"
            
            # Verificar extension
            supported_extensions = ['.csv', '.txt', '.kml', '.json']
            ext = file.suffix.lower()
            
            print("Archivo encontrado!")
            print(f"Nombre: {file.name}")
            print(f"Ubicacion: {file.parent}")
            print(f"Tamano: {size_str}")
            print(f"Extension: {ext}")
            
            if ext in supported_extensions:
                print(f"Formato compatible: {ext.upper()}")
            else:
                print(f"Extension no reconocida: {ext}")
                print("Se intentara procesar como archivo de texto")
            
            print()
            print("Listo para previsualizar en el siguiente paso!")
            
        except Exception as e:
            print(f"Error validando archivo: {e}")

# Conectar eventos
browse_button.on_click(lambda b: browse_file())
validate_button.on_click(lambda b: validate_file())

# Auto-validar cuando se cambie el texto
file_path_widget.observe(
    lambda change: validate_file() if change['name'] == 'value' and change['new'].strip() else None, 
    names='value'
)

display(widgets.VBox([
    widgets.HTML("<h4>Seleccionar archivo de datos</h4>"),
    file_path_widget,
    widgets.HBox([browse_button, validate_button]),
    file_validation_output
]))

## 👀 Paso 5: Previsualizar Datos

Antes de procesar, echemos un vistazo a tus datos. El sistema automáticamente detectará el formato y mostrará información relevante.

In [ ]:
# Widget mejorado para previsualizar datos desde cualquier ubicación
preview_button = widgets.Button(
    description='👀 Previsualizar Archivo',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

preview_output = widgets.Output()
loaded_data = None  # Variable global para almacenar datos cargados

def preview_file(b):
    global loaded_data
    
    with preview_output:
        clear_output()
        
        # Obtener ruta del archivo del widget mejorado
        file_path_str = file_path_widget.value.strip()
        if not file_path_str:
            print("❌ Por favor, especifica la ruta de un archivo")
            print("💡 Usa el explorador de arriba o escribe la ruta completa")
            return
            
        try:
            file_path = Path(file_path_str)
            
            if not file_path.exists():
                print(f"❌ El archivo no existe: {file_path_str}")
                print("💡 Verifica que la ruta sea correcta")
                return
                
            if not file_path.is_file():
                print(f"❌ La ruta no es un archivo válido: {file_path_str}")
                return
            
            print(f"📄 Previsualizando: {file_path.name}")
            print(f"📁 Ubicación: {file_path.parent}")
            print("=" * 70)
            
            # Detectar tipo de archivo por extensión y contenido
            file_extension = file_path.suffix.lower()
            
            if file_extension == '.csv':
                # Procesar archivo CSV
                try:
                    # Intentar diferentes encodings
                    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
                    df = None
                    used_encoding = None
                    
                    for encoding in encodings:
                        try:
                            df = pd.read_csv(file_path, encoding=encoding)
                            used_encoding = encoding
                            break
                        except UnicodeDecodeError:
                            continue
                    
                    if df is None:
                        print(f"❌ No se pudo leer el archivo CSV con ninguna codificación")
                        return
                    
                    loaded_data = {'type': 'csv', 'data': df, 'path': file_path, 'encoding': used_encoding}
                    
                    print(f"📊 Archivo CSV cargado exitosamente:")
                    print(f"  • Codificación: {used_encoding}")
                    print(f"  • Filas: {len(df):,}")
                    print(f"  • Columnas: {len(df.columns)}")
                    print(f"  • Columnas disponibles: {', '.join(df.columns)}")
                    
                    # Detectar columnas de coordenadas con más variaciones
                    lat_cols = []
                    lng_cols = []
                    
                    for col in df.columns:
                        col_lower = col.lower().strip()
                        # Patrones para latitud
                        if any(pattern in col_lower for pattern in ['lat', 'y', 'norte', 'north']):
                            lat_cols.append(col)
                        # Patrones para longitud
                        elif any(pattern in col_lower for pattern in ['lng', 'lon', 'long', 'x', 'oeste', 'east', 'west']):
                            lng_cols.append(col)
                    
                    if lat_cols and lng_cols:
                        print(f"\n🎯 Columnas de coordenadas detectadas:")
                        print(f"  • Latitud: {lat_cols[0]}")
                        print(f"  • Longitud: {lng_cols[0]}")
                        
                        # Validar que contienen números
                        try:
                            sample_lat = pd.to_numeric(df[lat_cols[0]], errors='coerce').dropna()
                            sample_lng = pd.to_numeric(df[lng_cols[0]], errors='coerce').dropna()
                            
                            if len(sample_lat) > 0 and len(sample_lng) > 0:
                                lat_range = f"{sample_lat.min():.4f} a {sample_lat.max():.4f}"
                                lng_range = f"{sample_lng.min():.4f} a {sample_lng.max():.4f}"
                                print(f"  • Rango latitud: {lat_range}")
                                print(f"  • Rango longitud: {lng_range}")
                                print(f"  • Coordenadas válidas: {min(len(sample_lat), len(sample_lng)):,}")
                            else:
                                print(f"  ⚠️  Las columnas no contienen coordenadas válidas")
                        except Exception:
                            print(f"  ⚠️  Error validando coordenadas")
                    else:
                        print(f"\n⚠️  No se detectaron columnas de coordenadas estándar")
                        print(f"💡 Columnas disponibles: {', '.join(df.columns)}")
                        print(f"💡 Asegúrate de que tengas columnas con nombres como 'lat', 'lng', 'latitude', 'longitude'")
                    
                    print(f"\n📋 Primeras 5 filas:")
                    try:
                        print(df.head().to_string(max_cols=10, max_colwidth=30))
                    except Exception:
                        print("(Error mostrando preview de datos)")
                        
                except Exception as e:
                    print(f"❌ Error procesando archivo CSV: {e}")
                    return
                    
            elif file_extension == '.kml':
                # Procesar archivo KML
                try:
                    extractor = KMLExtractor(str(file_path))
                    data = extractor.extract_data()
                    loaded_data = {'type': 'kml', 'data': data, 'path': file_path}
                    
                    print(f"🗺️  Archivo KML procesado:")
                    print(f"  • Placemarks encontrados: {len(data)}")
                    
                    with_coords = sum(1 for item in data if item['longitude'] is not None and item['latitude'] is not None)
                    with_ids = sum(1 for item in data if item['id'] is not None)
                    with_addresses = sum(1 for item in data if item['address'] is not None)
                    
                    print(f"  • Con coordenadas válidas: {with_coords}")
                    print(f"  • Con IDs: {with_ids}")
                    print(f"  • Con direcciones: {with_addresses}")
                    
                    if with_coords > 0:
                        # Calcular rangos de coordenadas
                        lats = [item['latitude'] for item in data if item['latitude'] is not None]
                        lngs = [item['longitude'] for item in data if item['longitude'] is not None]
                        
                        if lats and lngs:
                            lat_range = f"{min(lats):.4f} a {max(lats):.4f}"
                            lng_range = f"{min(lngs):.4f} a {max(lngs):.4f}"
                            print(f"  • Rango latitud: {lat_range}")
                            print(f"  • Rango longitud: {lng_range}")
                    
                    if data:
                        print(f"\n📋 Primeros 3 elementos:")
                        for i, item in enumerate(data[:3]):
                            print(f"\n  Elemento {i+1}:")
                            print(f"    ID: {item.get('id', 'N/A')}")
                            if item.get('longitude') and item.get('latitude'):
                                print(f"    Coordenadas: ({item['longitude']:.6f}, {item['latitude']:.6f})")
                            else:
                                print(f"    Coordenadas: No disponibles")
                            address = item.get('address', 'N/A')
                            if address and len(address) > 60:
                                address = address[:60] + "..."
                            print(f"    Dirección: {address}")
                    
                except Exception as e:
                    print(f"❌ Error procesando archivo KML: {e}")
                    return
                    
            elif file_extension == '.txt':
                # Procesar archivo TXT
                try:
                    # Intentar diferentes encodings
                    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
                    lines = None
                    used_encoding = None
                    
                    for encoding in encodings:
                        try:
                            with open(file_path, 'r', encoding=encoding) as f:
                                lines = f.readlines()
                            used_encoding = encoding
                            break
                        except UnicodeDecodeError:
                            continue
                    
                    if lines is None:
                        print(f"❌ No se pudo leer el archivo TXT con ninguna codificación")
                        return
                        
                    loaded_data = {'type': 'txt', 'data': lines, 'path': file_path, 'encoding': used_encoding}
                    
                    print(f"📝 Archivo TXT cargado:")
                    print(f"  • Codificación: {used_encoding}")
                    print(f"  • Total de líneas: {len(lines):,}")
                    
                    # Mostrar primeras líneas
                    print(f"  • Primeras líneas del archivo:")
                    for i, line in enumerate(lines[:5]):
                        line_preview = line.strip()
                        if len(line_preview) > 60:
                            line_preview = line_preview[:60] + "..."
                        print(f"    {i+1}: {line_preview}")
                    
                    # Analizar formato de coordenadas
                    coord_count = 0
                    valid_coords = []
                    
                    for line_num, line in enumerate(lines[:100]):  # Analizar las primeras 100 líneas
                        line = line.strip()
                        if not line or line.startswith('#'):
                            continue
                            
                        # Probar diferentes separadores
                        for separator in [',', ';', '\t', ' ']:
                            parts = line.split(separator)
                            if len(parts) >= 2:
                                try:
                                    lat = float(parts[0].strip())
                                    lng = float(parts[1].strip())
                                    # Validar rangos aproximados de coordenadas
                                    if -90 <= lat <= 90 and -180 <= lng <= 180:
                                        coord_count += 1
                                        valid_coords.append((lat, lng))
                                        break
                                except ValueError:
                                    pass
                    
                    print(f"\n🎯 Análisis de coordenadas:")
                    print(f"  • Líneas con coordenadas válidas: {coord_count}")
                    
                    if valid_coords:
                        lats = [coord[0] for coord in valid_coords]
                        lngs = [coord[1] for coord in valid_coords]
                        lat_range = f"{min(lats):.4f} a {max(lats):.4f}"
                        lng_range = f"{min(lngs):.4f} a {max(lngs):.4f}"
                        print(f"  • Rango latitud: {lat_range}")
                        print(f"  • Rango longitud: {lng_range}")
                        print(f"  • Formato detectado: Separado por {', ' if ',' in lines[0] else 'espacios'}")
                    else:
                        print(f"  ⚠️  No se detectaron coordenadas válidas en las primeras 100 líneas")
                        print(f"  💡 Asegúrate de que el formato sea: latitud,longitud o latitud longitud")
                    
                except Exception as e:
                    print(f"❌ Error procesando archivo TXT: {e}")
                    return
                    
            elif file_extension == '.json':
                # Procesar archivo JSON
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        json_data = json.load(f)
                    
                    loaded_data = {'type': 'json', 'data': json_data, 'path': file_path}
                    
                    print(f"📋 Archivo JSON cargado:")
                    if isinstance(json_data, list):
                        print(f"  • Tipo: Lista con {len(json_data)} elementos")
                        if json_data:
                            print(f"  • Estructura del primer elemento: {list(json_data[0].keys()) if isinstance(json_data[0], dict) else 'No es diccionario'}")
                    elif isinstance(json_data, dict):
                        print(f"  • Tipo: Diccionario con {len(json_data)} claves")
                        print(f"  • Claves principales: {list(json_data.keys())}")
                    
                    # Intentar detectar coordenadas en JSON
                    coord_fields = ['lat', 'lng', 'latitude', 'longitude', 'coordinates']
                    print(f"\n🎯 Buscando campos de coordenadas...")
                    # Implementación básica - se puede expandir según necesidades
                    
                except Exception as e:
                    print(f"❌ Error procesando archivo JSON: {e}")
                    return
            else:
                print(f"⚠️  Formato de archivo '{file_extension}' no reconocido")
                print(f"💡 Formatos soportados: .csv, .txt, .kml, .json")
                print(f"🔄 Se intentará procesar como archivo de texto...")
                
                # Intentar como archivo de texto genérico
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read(1000)  # Leer los primeros 1000 caracteres
                    print(f"\n📄 Vista previa del contenido:")
                    print(content[:500] + "..." if len(content) > 500 else content)
                except Exception as e:
                    print(f"❌ No se pudo leer el archivo: {e}")
                    return
            
            print(f"\n✅ Archivo cargado correctamente")
            print(f"💾 Tamaño: {file_path.stat().st_size:,} bytes")
            print(f"📅 Modificado: {datetime.fromtimestamp(file_path.stat().st_mtime).strftime('%Y-%m-%d %H:%M')}")
            print(f"\n💡 ¡Listo! Ahora puedes continuar con el procesamiento en las siguientes celdas.")
            
        except Exception as e:
            print(f"❌ Error leyendo archivo: {e}")
            print(f"💡 Verifica que:")
            print(f"  • El archivo existe y es accesible")
            print(f"  • Tienes permisos para leerlo")
            print(f"  • La ruta está escrita correctamente")
            print(f"  • El archivo no está abierto en otro programa")
            loaded_data = None

preview_button.on_click(preview_file)

display(widgets.VBox([
    preview_button,
    preview_output
]))

## ⚙️ Paso 6: Configurar Opciones de Procesamiento

Personaliza cómo quieres procesar tus datos y **dónde guardar** los resultados.

In [ ]:
# Configuración simplificada y rápida
print("🔧 Configuración de Procesamiento")
print("Personaliza las opciones según tus necesidades:\n")

# Configuración de idioma
language_widget = widgets.Dropdown(
    options=[('Español', 'es'), ('English', 'en'), ('Français', 'fr'), ('Deutsch', 'de'), ('Italiano', 'it')],
    value='es',
    description='Idioma:',
    style={'description_width': 'initial'}
)

# Configuración de país
country_widget = widgets.Dropdown(
    options=[
        ('Automático', ''),
        ('España', 'es'),
        ('Estados Unidos', 'us'),
        ('México', 'mx'),
        ('Argentina', 'ar'),
        ('Colombia', 'co'),
        ('Francia', 'fr'),
        ('Alemania', 'de'),
        ('Reino Unido', 'gb')
    ],
    value='',
    description='País preferido:',
    style={'description_width': 'initial'}
)

# Configuración de limpieza
clean_widget = widgets.Checkbox(
    value=False,
    description='Limpiar caracteres especiales (?, !, @, etc.)',
    style={'description_width': 'initial'}
)

aggressive_widget = widgets.Checkbox(
    value=False,
    description='Limpieza agresiva (también quita acentos)',
    style={'description_width': 'initial'}
)

# Configuración de velocidad
delay_widget = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=5.0,
    step=0.1,
    description='Retraso entre consultas (seg):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Configuración de salida simplificada
output_format_widget = widgets.SelectMultiple(
    options=[('CSV', 'csv'), ('JSON', 'json'), ('Excel', 'xlsx')],
    value=['csv'],
    description='Formatos de salida:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(height='80px')
)

output_name_widget = widgets.Text(
    value='resultados_geocoding',
    description='Nombre archivo:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Configuración de directorio de salida simplificada
output_dir_widget = widgets.Text(
    value=str(Path.home() / "Desktop"),  # Por defecto al escritorio
    placeholder='Ejemplo: C:\\Users\\Usuario\\OneDrive\\Documentos',
    description='Carpeta destino:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

browse_output_button = widgets.Button(
    description='📁 Explorar',
    button_style='info',
    layout=widgets.Layout(width='100px')
)

output_validation = widgets.Output()

def browse_output_directory():
    """Seleccionar directorio de salida"""
    def dir_dialog():
        try:
            import tkinter as tk
            from tkinter import filedialog
            
            root = tk.Tk()
            root.withdraw()
            root.lift()
            root.attributes('-topmost', True)
            
            directory = filedialog.askdirectory(
                title="Seleccionar carpeta para guardar resultados"
            )
            
            if directory:
                output_dir_widget.value = directory
                validate_output_dir()
            
            root.destroy()
            
        except Exception as e:
            with output_validation:
                clear_output()
                print(f"❌ Error: {e}")
    
    thread = threading.Thread(target=dir_dialog)
    thread.daemon = True
    thread.start()

def validate_output_dir():
    """Validar directorio de salida (rápido)"""
    with output_validation:
        clear_output()
        
        dir_path = output_dir_widget.value.strip()
        if not dir_path:
            return
            
        try:
            directory = Path(dir_path)
            
            if directory.exists() and directory.is_dir():
                print(f"✅ Carpeta válida: {directory}")
            else:
                print(f"💡 Se creará la carpeta: {directory}")
            
        except Exception as e:
            print(f"⚠️  Verifica la ruta: {e}")

# Conectar eventos
browse_output_button.on_click(lambda b: browse_output_directory())
output_dir_widget.observe(lambda change: validate_output_dir() if change['name'] == 'value' else None, names='value')

# Validar directorio inicial
validate_output_dir()

# Organizar widgets de forma más compacta
config_widgets = widgets.VBox([
    widgets.HTML("<h4>🌐 Configuración Regional</h4>"),
    widgets.HBox([language_widget, country_widget]),
    
    widgets.HTML("<br><h4>🧹 Opciones de Limpieza</h4>"),
    clean_widget,
    aggressive_widget,
    
    widgets.HTML("<br><h4>⚡ Velocidad</h4>"),
    delay_widget,
    widgets.HTML("<small>💡 Menor retraso = más rápido</small>"),
    
    widgets.HTML("<br><h4>💾 Archivos de Salida</h4>"),
    widgets.HBox([output_format_widget, widgets.VBox([output_name_widget, widgets.HBox([output_dir_widget, browse_output_button])])]),
    output_validation
])

display(config_widgets)

print("\n✅ Configuración lista. Continúa con el siguiente paso.")

## 🚀 Paso 7: Procesar Datos

¡Hora de procesar tus datos! Este paso realizará la geocodificación y limpieza.

In [ ]:
# Botón de procesamiento principal
process_button = widgets.Button(
    description='🚀 Procesar Datos',
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px'),
    style={'font_weight': 'bold'}
)

process_output = widgets.Output()
processed_results = None  # Variable global para resultados

def process_data(b):
    global processed_results
    
    with process_output:
        clear_output()
        
        if loaded_data is None:
            print("❌ Primero debes cargar un archivo en el paso anterior")
            return
            
        # Verificar API key
        if 'OPENCAGE_API_KEY' not in os.environ:
            print("❌ Primero configura tu API key en el Paso 3")
            return
            
        try:
            print("🚀 Iniciando procesamiento...")
            print("=" * 50)
            
            # Obtener configuración
            language = language_widget.value
            country = country_widget.value if country_widget.value else None
            clean_chars = clean_widget.value
            aggressive = aggressive_widget.value
            delay = delay_widget.value
            
            print(f"📋 Configuración:")
            print(f"  • Idioma: {language}")
            print(f"  • País preferido: {country or 'Automático'}")
            print(f"  • Limpieza: {'Agresiva' if aggressive else 'Normal' if clean_chars else 'Ninguna'}")
            print(f"  • Retraso: {delay} segundos")
            print()
            
            # Inicializar geocoder
            from opencage.geocoder import OpenCageGeocode
            geocoder = OpenCageGeocode(os.environ['OPENCAGE_API_KEY'])
            
            results = []
            
            if loaded_data['type'] == 'csv':
                print("📊 Procesando archivo CSV...")
                df = loaded_data['data']
                
                # Detectar columnas de coordenadas
                lat_col = None
                lng_col = None
                
                for col in df.columns:
                    col_lower = col.lower()
                    if 'lat' in col_lower and lat_col is None:
                        lat_col = col
                    elif any(x in col_lower for x in ['lng', 'lon']) and lng_col is None:
                        lng_col = col
                
                if lat_col is None or lng_col is None:
                    print("❌ No se pudieron detectar columnas de coordenadas")
                    print(f"Columnas disponibles: {', '.join(df.columns)}")
                    return
                    
                print(f"🎯 Usando columnas: {lat_col}, {lng_col}")
                
                # Procesar coordenadas
                coordinates = []
                for idx, row in df.iterrows():
                    try:
                        lat = float(row[lat_col])
                        lng = float(row[lng_col])
                        coordinates.append((lat, lng))
                    except (ValueError, TypeError):
                        print(f"⚠️  Saltando fila {idx + 1}: coordenadas inválidas")
                        continue
                
                print(f"📍 Procesando {len(coordinates)} coordenadas...")
                results = reverse_geocode_coordinates(
                    geocoder, coordinates, delay, clean_chars, aggressive, language, country
                )
                
            elif loaded_data['type'] == 'kml':
                print("🗺️  Procesando archivo KML...")
                kml_data = loaded_data['data']
                
                coordinates = []
                for item in kml_data:
                    if item['latitude'] is not None and item['longitude'] is not None:
                        coordinates.append((item['latitude'], item['longitude']))
                
                print(f"📍 Procesando {len(coordinates)} coordenadas del KML...")
                results = reverse_geocode_coordinates(
                    geocoder, coordinates, delay, clean_chars, aggressive, language, country
                )
                
                # Añadir información del KML original
                for i, result in enumerate(results):
                    if i < len(kml_data):
                        result['original_id'] = kml_data[i].get('id')
                        result['original_address'] = kml_data[i].get('address')
                
            elif loaded_data['type'] == 'txt':
                print("📝 Procesando archivo TXT...")
                
                # Leer todas las líneas del archivo
                with open(loaded_data['path'], 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                
                coordinates = []
                for line_num, line in enumerate(lines, 1):
                    line = line.strip()
                    if not line or line.startswith('#'):
                        continue
                        
                    try:
                        if ',' in line:
                            parts = line.split(',')
                        else:
                            parts = line.split()
                        
                        if len(parts) >= 2:
                            lat = float(parts[0].strip())
                            lng = float(parts[1].strip())
                            coordinates.append((lat, lng))
                    except (ValueError, IndexError):
                        print(f"⚠️  Saltando línea {line_num}: formato inválido")
                        continue
                
                print(f"📍 Procesando {len(coordinates)} coordenadas...")
                results = reverse_geocode_coordinates(
                    geocoder, coordinates, delay, clean_chars, aggressive, language, country
                )
            
            # Guardar resultados
            processed_results = results
            
            print("\n" + "=" * 50)
            print(f"✅ ¡Procesamiento completado!")
            print(f"📊 Resultados: {len(results)} ubicaciones procesadas")
            
            # Estadísticas
            successful = sum(1 for r in results if 'Error:' not in r.get('address', ''))
            failed = len(results) - successful
            
            print(f"  • Exitosas: {successful}")
            print(f"  • Fallidas: {failed}")
            
            if results:
                print(f"\n📋 Muestra de resultados:")
                for i, result in enumerate(results[:3]):
                    print(f"\n  Resultado {i+1}:")
                    print(f"    Coordenadas: ({result['latitude']}, {result['longitude']})")
                    print(f"    Dirección: {result['address'][:60]}...")
                    print(f"    País: {result.get('country', 'N/A')}")
            
            print("\n💡 Continúa con el siguiente paso para exportar los resultados.")
            
        except Exception as e:
            print(f"❌ Error durante el procesamiento: {e}")
            import traceback
            traceback.print_exc()

process_button.on_click(process_data)

display(widgets.VBox([
    process_button,
    process_output
]))

## 💾 Paso 8: Exportar Resultados

Guarda tus resultados **en la ubicación que elijas** y en los formatos que necesites.

In [ ]:
# Botón de exportación optimizado
export_button = widgets.Button(
    description='💾 Exportar Resultados',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

export_output = widgets.Output()

def export_results(b):
    with export_output:
        clear_output()
        
        if processed_results is None:
            print("❌ Primero debes procesar datos en el paso anterior")
            return
            
        try:
            # Obtener configuración de salida (usando widgets del Paso 6)
            output_name = output_name_widget.value.strip() or 'resultados_geocoding'
            output_dir = output_dir_widget.value.strip() or str(Path.cwd())
            formats = output_format_widget.value
            
            # Validar directorio de salida
            output_directory = Path(output_dir)
            output_directory.mkdir(parents=True, exist_ok=True)
            
            print(f"💾 Exportando {len(processed_results)} registros...")
            print(f"📁 Destino: {output_directory}")
            print(f"📄 Formatos: {', '.join(formats)}")
            print()
            
            exported_files = []
            
            # Exportar en los formatos seleccionados
            for format_type in formats:
                try:
                    if format_type == 'csv':
                        filename = output_directory / f"{output_name}.csv"
                        df = pd.DataFrame(processed_results)
                        df.to_csv(filename, index=False, encoding='utf-8')
                        exported_files.append(filename)
                        print(f"✅ CSV: {filename.name}")
                        
                    elif format_type == 'json':
                        filename = output_directory / f"{output_name}.json"
                        with open(filename, 'w', encoding='utf-8') as f:
                            json.dump(processed_results, f, indent=2, ensure_ascii=False)
                        exported_files.append(filename)
                        print(f"✅ JSON: {filename.name}")
                        
                    elif format_type == 'xlsx':
                        filename = output_directory / f"{output_name}.xlsx"
                        df = pd.DataFrame(processed_results)
                        
                        try:
                            df.to_excel(filename, index=False, engine='openpyxl')
                            exported_files.append(filename)
                            print(f"✅ Excel: {filename.name}")
                        except ImportError:
                            # Fallback a CSV con separador de punto y coma
                            filename_csv = output_directory / f"{output_name}_excel.csv"
                            df.to_csv(filename_csv, index=False, encoding='utf-8', sep=';')
                            exported_files.append(filename_csv)
                            print(f"✅ CSV (Excel): {filename_csv.name}")
                        
                except Exception as e:
                    print(f"❌ Error con {format_type}: {str(e)[:50]}")
            
            # Resumen final
            if exported_files:
                print(f"\n🎉 Exportación completada!")
                print(f"📂 {len(exported_files)} archivo(s) creado(s)")
                print(f"📍 Ubicación: {output_directory}")
                
                for filename in exported_files:
                    print(f"   • {filename.name}")
                
                print(f"\n💡 Los archivos están listos para usar.")
                
            else:
                print("❌ No se pudo exportar ningún archivo")
                
        except Exception as e:
            print(f"❌ Error durante la exportación: {e}")

export_button.on_click(export_results)

display(widgets.VBox([
    export_button,
    export_output
]))

## ? Paso 7: Procesar Datos

¡Hora de procesar tus datos! El sistema automáticamente trabajará con el archivo que cargaste desde cualquier ubicación.

In [ ]:
# Botón para generar reporte
report_button = widgets.Button(
    description='📊 Generar Reporte',
    button_style='info',
    layout=widgets.Layout(width='200px', height='40px')
)

report_output = widgets.Output()

def generate_report(b):
    with report_output:
        clear_output()
        
        if processed_results is None:
            print("❌ Primero debes procesar datos")
            return
            
        try:
            print("📊 Generando reporte de calidad...")
            print("=" * 60)
            
            total = len(processed_results)
            successful = sum(1 for r in processed_results if 'Error:' not in r.get('address', ''))
            failed = total - successful
            
            print(f"📈 ESTADÍSTICAS GENERALES")
            print(f"  • Total de coordenadas procesadas: {total:,}")
            print(f"  • Geocodificación exitosa: {successful:,} ({successful/total*100:.1f}%)")
            print(f"  • Geocodificación fallida: {failed:,} ({failed/total*100:.1f}%)")
            
            if successful > 0:
                # Análisis de países
                countries = {}
                for result in processed_results:
                    if 'Error:' not in result.get('address', ''):
                        country = result.get('country', 'Desconocido')
                        countries[country] = countries.get(country, 0) + 1
                
                print(f"\n🌍 DISTRIBUCIÓN POR PAÍSES")
                sorted_countries = sorted(countries.items(), key=lambda x: x[1], reverse=True)
                for country, count in sorted_countries[:10]:  # Top 10 países
                    percentage = count / successful * 100
                    print(f"  • {country}: {count:,} ({percentage:.1f}%)")
                
                # Análisis de componentes de dirección
                has_state = sum(1 for r in processed_results if r.get('state'))
                has_city = sum(1 for r in processed_results if r.get('city'))
                has_postcode = sum(1 for r in processed_results if r.get('postcode'))
                
                print(f"\n🏢 COMPLETITUD DE DATOS")
                print(f"  • Con estado/provincia: {has_state:,} ({has_state/total*100:.1f}%)")
                print(f"  • Con ciudad: {has_city:,} ({has_city/total*100:.1f}%)")
                print(f"  • Con código postal: {has_postcode:,} ({has_postcode/total*100:.1f}%)")
                
                # Crear gráfico de calidad
                try:
                    import plotly.graph_objects as go
                    from plotly.subplots import make_subplots
                    
                    # Gráfico de barras de éxito/falla
                    fig = make_subplots(
                        rows=2, cols=2,
                        subplot_titles=(
                            'Tasa de Éxito de Geocodificación',
                            'Top 5 Países',
                            'Completitud de Datos',
                            'Distribución de Calidad'
                        ),
                        specs=[[{'type': 'bar'}, {'type': 'bar'}],
                               [{'type': 'bar'}, {'type': 'pie'}]]
                    )
                    
                    # Gráfico 1: Éxito vs Falla
                    fig.add_trace(
                        go.Bar(x=['Exitoso', 'Fallido'], y=[successful, failed],
                               marker_color=['green', 'red']),
                        row=1, col=1
                    )
                    
                    # Gráfico 2: Top países
                    top_countries = sorted_countries[:5]
                    if top_countries:
                        countries_names = [c[0] for c in top_countries]
                        countries_counts = [c[1] for c in top_countries]
                        fig.add_trace(
                            go.Bar(x=countries_names, y=countries_counts,
                                   marker_color='blue'),
                            row=1, col=2
                        )
                    
                    # Gráfico 3: Completitud
                    fig.add_trace(
                        go.Bar(x=['Estado', 'Ciudad', 'Código Postal'],
                               y=[has_state, has_city, has_postcode],
                               marker_color='orange'),
                        row=2, col=1
                    )
                    
                    # Gráfico 4: Distribución general
                    labels = ['Datos Completos', 'Datos Parciales', 'Sin Datos']
                    complete = sum(1 for r in processed_results 
                                 if r.get('state') and r.get('city') and r.get('postcode'))
                    partial = successful - complete
                    no_data = failed
                    
                    fig.add_trace(
                        go.Pie(labels=labels, values=[complete, partial, no_data],
                               marker_colors=['green', 'yellow', 'red']),
                        row=2, col=2
                    )
                    
                    fig.update_layout(
                        height=800,
                        title_text="📊 Reporte de Calidad de Geocodificación",
                        showlegend=False
                    )
                    
                    fig.show()
                    
                except ImportError:
                    print("\n📊 Para ver gráficos, instala plotly: pip install plotly")
                except Exception as e:
                    print(f"\n⚠️  Error generando gráficos: {e}")
                
                print(f"\n✅ Reporte generado exitosamente")
                
                # Sugerencias de mejora
                print(f"\n💡 SUGERENCIAS")
                if failed > total * 0.1:  # Más del 10% falló
                    print(f"  • Alta tasa de falla ({failed/total*100:.1f}%). Revisa la calidad de las coordenadas.")
                if has_postcode < successful * 0.5:  # Menos del 50% tiene código postal
                    print(f"  • Pocos códigos postales encontrados. Considera usar un país específico.")
                if successful > 0:
                    print(f"  • Para mejorar resultados, usa configuración de país específico.")
                    print(f"  • Considera ajustar el idioma según tu región de interés.")
            
        except Exception as e:
            print(f"❌ Error generando reporte: {e}")
            import traceback
            traceback.print_exc()

report_button.on_click(generate_report)

display(widgets.VBox([
    report_button,
    report_output
]))

## 🗺️ Paso 10: Visualizar en Mapa (Opcional)

Crea un mapa interactivo con tus resultados geocodificados.

In [ ]:
# Botón para crear mapa
map_button = widgets.Button(
    description='🗺️ Crear Mapa Interactivo',
    button_style='warning',
    layout=widgets.Layout(width='250px', height='40px')
)

map_output = widgets.Output()

def create_map(b):
    with map_output:
        clear_output()
        
        if processed_results is None:
            print("❌ Primero debes procesar datos")
            return
            
        try:
            print("🗺️ Creando mapa interactivo...")
            
            # Filtrar solo resultados exitosos
            valid_results = [
                r for r in processed_results 
                if 'Error:' not in r.get('address', '') and 
                   r.get('latitude') is not None and 
                   r.get('longitude') is not None
            ]
            
            if not valid_results:
                print("❌ No hay resultados válidos para mapear")
                return
                
            print(f"📍 Mostrando {len(valid_results)} ubicaciones en el mapa")
            
            # Intentar usar plotly para el mapa
            try:
                import plotly.express as px
                
                # Preparar datos para el mapa
                df_map = pd.DataFrame(valid_results)
                
                # Crear mapa con plotly
                fig = px.scatter_mapbox(
                    df_map,
                    lat='latitude',
                    lon='longitude',
                    hover_name='address',
                    hover_data={'country': True, 'city': True, 'latitude': ':.4f', 'longitude': ':.4f'},
                    zoom=3,
                    height=600,
                    title=f"🗺️ Mapa de {len(valid_results)} Ubicaciones Geocodificadas"
                )
                
                fig.update_layout(
                    mapbox_style="open-street-map",
                    margin={"r":0,"t":50,"l":0,"b":0}
                )
                
                fig.show()
                
                print("✅ Mapa creado exitosamente")
                print("💡 Puedes hacer zoom, arrastrar y hacer clic en los puntos para más información")
                
            except ImportError:
                print("⚠️  Plotly no está disponible. Intentando método alternativo...")
                
                # Método alternativo: mostrar coordenadas en tabla
                print("\n📋 Ubicaciones procesadas:")
                print("=" * 80)
                
                for i, result in enumerate(valid_results[:20]):  # Mostrar solo las primeras 20
                    lat = result['latitude']
                    lng = result['longitude']
                    address = result['address'][:50] + '...' if len(result['address']) > 50 else result['address']
                    country = result.get('country', 'N/A')
                    
                    print(f"{i+1:2d}. {lat:8.4f}, {lng:9.4f} | {country:15s} | {address}")
                
                if len(valid_results) > 20:
                    print(f"\n... y {len(valid_results) - 20} ubicaciones más")
                
                print(f"\n💡 Para ver un mapa interactivo, instala plotly: pip install plotly")
                
        except Exception as e:
            print(f"❌ Error creando mapa: {e}")
            import traceback
            traceback.print_exc()

map_button.on_click(create_map)

display(widgets.VBox([
    map_button,
    map_output
]))

## 🎉 ¡Felicidades!

Has completado el procesamiento de geocodificación. 

### 📋 Resumen de lo que hiciste:
1. ✅ Instalaste las dependencias necesarias
2. ✅ Configuraste tu API key de OpenCage
3. ✅ Cargaste y previsualizaste tus datos
4. ✅ Configuraste las opciones de procesamiento
5. ✅ Procesaste los datos con geocodificación
6. ✅ Exportaste los resultados
7. ✅ Generaste reportes de calidad
8. ✅ (Opcional) Creaste visualizaciones

### 🔄 Para procesar más archivos:
- Simplemente regresa al **Paso 4** y carga un nuevo archivo
- Ajusta la configuración en el **Paso 6** si es necesario
- Ejecuta de nuevo desde el **Paso 7**

### 📞 ¿Necesitas ayuda?
- Revisa que tu API key de OpenCage sea válida
- Verifica que tus archivos tengan el formato correcto
- Asegúrate de que las coordenadas estén en formato decimal (ej: 40.7128, -74.0060)

### 🛠️ Funciones Avanzadas:
Si necesitas funciones más avanzadas, puedes usar directamente las herramientas en modo script:
- `reverse_geocoding_batch.py` - Para procesamiento por lotes
- `kml_extractor.py` - Para extraer datos de KML
- `address_enhancer.py` - Para limpieza avanzada de direcciones

---

<div style="background-color: #e8f5e8; padding: 20px; border-radius: 10px; border-left: 5px solid #4CAF50;">
    <h3>🎯 ¡Misión Cumplida!</h3>
    <p>Has procesado exitosamente tus datos geográficos. Los archivos están listos para usar en tu proyecto.</p>
    <p><strong>Recuerda:</strong> OpenCage ofrece 2,500 consultas gratuitas por día. Para más volumen, considera una cuenta premium.</p>
</div>